In [1]:
import scipy.io
import sys
import os
import torch
import torch.nn as nn
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'../..')))

from Rmse_loss import rmse_loss 
from All import all0
from Select import select
from Train_fun import train_fun
from Random_choice import random_choice

torch.set_default_dtype(torch.float32)
device = torch.device('cuda:0')
np.set_printoptions(suppress=True)

In [2]:
class FourNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64,64)
        self.linear4 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = self.linear4(x)
        return x

class FiveNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 64)
        self.linear2 = nn.Linear(64, 128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64,64)
        self.linear5 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = torch.sin(self.linear4(x))
        x = self.linear5(x)
        return x

In [3]:
def diff_NS(net,cor_train):
    i = 0
    cache = np.zeros((1,7))
    while i*10000<len(cor_train): 
        pts = cor_train[i*10000:(i+1)*10000].clone().detach().to(device)
        pts.requires_grad_(True)
        outs1 = net(pts)
        grad = torch.autograd.grad(outs1, pts, grad_outputs=torch.ones_like(outs1), create_graph=True)[0]
        wt_t = grad[:,0].reshape(-1,1) 
        wx_t = grad[:,1].reshape(-1,1)
        wy_t = grad[:,2].reshape(-1,1)
        wx = wx_t.cpu().detach().numpy()
        wy = wy_t.cpu().detach().numpy()
        wt = wt_t.cpu().detach().numpy()
        w_pred = outs1.cpu().detach().numpy()
        del wx_t, wt_t, wy_t, grad, outs1
        torch.cuda.empty_cache()

        outs1 = net(pts)
        grad = torch.autograd.grad(outs1, pts, grad_outputs=torch.ones_like(outs1), create_graph=True)[0]
        wxx_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs1), create_graph=True)[0])[:,1].reshape(-1,1)
        wxy_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs1), create_graph=True)[0])[:,2].reshape(-1,1)    
        wxx = wxx_t.cpu().detach().numpy()
        wxy = wxy_t.cpu().detach().numpy()
        del wxx_t, wxy_t

        wyy_t= (torch.autograd.grad(grad[:,2].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs1), create_graph=True)[0])[:,2].reshape(-1,1)
        wyy = wyy_t.cpu().detach().numpy()
        del wyy_t, grad, outs1
        torch.cuda.empty_cache()
        cache = np.vstack((cache,np.hstack((w_pred,wx,wy,wxx,wxy,wyy,wt))))
        i=i+1
    return cache[1:]

In [9]:
data = scipy.io.loadmat('Vorticity_ALL.mat')

In [71]:
steps = 151
n = 449
m = 199
dt = 0.2
dx = 0.02
dy = 0.02

W = data['VORTALL'].reshape(n,m,steps)   # vorticity
U = data['UALL'].reshape(n,m,steps)      # x-component of velocity
V = data['VALL'].reshape(n,m,steps)      # y-component of velocity

# Cut out the portion of the data before the cylinder
xmin = 100
xmax = 400
ymin = 15
ymax = 185        
W = W[xmin:xmax,ymin:ymax,:]
U = U[xmin:xmax,ymin:ymax,:]
V = V[xmin:xmax,ymin:ymax,:]
n,m,steps = W.shape

# Preprocess data #1(First dimensiton is space and the second dimension is time. Y varies first in the first dimension)
w_data = W.reshape(n*m, steps)
u_data = U.reshape(n*m, steps)
v_data = V.reshape(n*m, steps)

t_data = np.arange(steps).reshape((1, -1))*dt         
t_data = np.tile(t_data,(m*n,1))

# This part reset the coordinates
x_data = np.arange(n).reshape((-1, 1))*dx 
x_data = np.tile(x_data, (1, m))
x_data = np.reshape(x_data, (-1, 1))
x_data = np.tile(x_data, (1, steps))

y_data = np.arange(m).reshape((1, -1))*dy 
y_data = np.tile(y_data, (n, 1))
y_data = np.reshape(y_data, (-1, 1))
y_data = np.tile(y_data, (1, steps))

In [72]:
t_star = np.reshape(t_data,(-1,1))
x_star = np.reshape(x_data,(-1,1))
y_star = np.reshape(y_data,(-1,1))        
u_star = np.reshape(u_data,(-1,1))
v_star = np.reshape(v_data,(-1,1))
w_star = np.reshape(w_data,(-1,1))

In [73]:
data = np.hstack((t_star,x_star,y_star,u_star,v_star,w_star))
data = data[data[:,0]/0.2%4==0]
data = data[data[:,1]/0.02%4==0]
data = data[data[:,2]/0.02%4==0]
data = data.astype('float32')

In [82]:
per = 0.2

np.random.seed(0)
cor_train = torch.from_numpy(data[:,[0,1,2]])

w_clean = data[:,-1].reshape(-1,1)
w_noise = w_clean + per*np.std(w_clean)*np.random.randn(w_clean.shape[0],w_clean.shape[1])
w_noise = w_noise.astype('float32')
w_train = torch.from_numpy(w_noise)
n_variables = cor_train.shape[1]

u_clean = data[:,3].reshape(-1,1)
u_noise = u_clean + per*np.std(u_clean)*np.random.randn(u_clean.shape[0],u_clean.shape[1])
u_noise = u_noise.astype('float32')
u_train = torch.from_numpy(u_noise)

v_clean = data[:,4].reshape(-1,1)
v_noise = v_clean + per*np.std(v_clean)*np.random.randn(v_clean.shape[0],v_clean.shape[1])
v_noise = v_noise.astype('float32')
v_train = torch.from_numpy(v_noise)

In [83]:
netw = FourNet_sin(n_variables).to(device)
try:
    netw.load_state_dict(torch.load('nets/netw_{}'.format(per)))
except:
    train_fun(netw,cor_train,w_train,N_red_lr=4, epochs=1000,lr=0.01,threshold=0.001)
    torch.save(netw.state_dict(),'net/netw_{}'.format(per))

0 1.014012098312378
1 1.0555496215820312
2 1.0624786615371704
3 1.0026484727859497
4 0.9864567518234253
5 0.9839897751808167
6 0.9760690927505493
7 0.9727517366409302
8 0.9688705801963806
9 0.9634420871734619
10 0.9648410677909851
11 0.9642605781555176
12 0.9583526253700256
13 0.9541897773742676
14 0.9531598091125488
15 0.9521673321723938
16 0.9503105282783508
17 0.9477200508117676
18 0.943306565284729
19 0.9374972581863403
20 0.9320734739303589
21 0.9265336394309998
22 0.9190856218338013
23 0.9092389941215515
24 0.8966903686523438
25 0.8807135820388794
26 0.8613001108169556
27 0.8392347097396851
28 0.8131732940673828
29 0.7834944128990173
30 0.7534937262535095
31 0.725562572479248
32 0.7039200067520142
33 0.7161031365394592
34 0.7848381996154785
35 0.6507815718650818
36 0.698774516582489
37 0.614154040813446
38 0.6354056596755981
39 0.5968989729881287
40 0.586503803730011
41 0.5816970467567444
42 0.547248125076294
43 0.5483697652816772
44 0.5337882041931152
45 0.5124369263648987
46 0.

370 0.21243689954280853
371 0.21348616480827332
372 0.21330149471759796
373 0.21199919283390045
374 0.2098366618156433
375 0.20754416286945343
376 0.20584431290626526
377 0.20490762591362
378 0.20497702062129974
379 0.20605109632015228
380 0.20787476003170013
381 0.20964333415031433
382 0.21042020618915558
383 0.2098245471715927
384 0.2086099535226822
385 0.2073850780725479
386 0.20669697225093842
387 0.20614886283874512
388 0.20572751760482788
389 0.20519888401031494
390 0.20476914942264557
391 0.20446902513504028
392 0.20438818633556366
393 0.20454992353916168
394 0.20497839152812958
395 0.20573323965072632
396 0.2067863494157791
397 0.20810329914093018
398 0.2093648761510849
399 0.2103656828403473
0.21140951037406921 0.20826266437768937 0.014885073007414857
400 0.2107904851436615
401 0.21069543063640594
402 0.21006013453006744
403 0.20918849110603333
404 0.20851390063762665
405 0.20827297866344452
406 0.20899423956871033
407 0.20990972220897675
408 0.21083898842334747
409 0.21020808

742 0.19809183478355408
743 0.19858241081237793
744 0.19931039214134216
745 0.20016123354434967
746 0.2012529969215393
747 0.20201723277568817
748 0.2025832086801529
749 0.2021632045507431
0.2005699273943901 0.19989120036363603 0.003383992005040072
750 0.20139937102794647
751 0.2001468390226364
752 0.19908937811851501
753 0.19825975596904755
754 0.1978304088115692
755 0.1978030800819397
756 0.19823886454105377
757 0.1992797702550888
758 0.20100095868110657
759 0.2033594846725464
760 0.20544618368148804
761 0.20609867572784424
762 0.20466814935207367
763 0.20208129286766052
764 0.1997973769903183
765 0.19833043217658997
766 0.19771134853363037
767 0.1975996047258377
768 0.1978238821029663
769 0.19809526205062866
770 0.19841189682483673
771 0.1985914409160614
772 0.19877713918685913
773 0.19897165894508362
774 0.19927290081977844
775 0.19968806207180023
776 0.2000308632850647
777 0.20026466250419617
778 0.20002612471580505
779 0.19957023859024048
780 0.1987345963716507
781 0.198003515601

62 0.19542138278484344
63 0.19542062282562256
64 0.19541983306407928
65 0.1954190582036972
66 0.19541826844215393
67 0.19541747868061066
68 0.19541670382022858
69 0.1954159140586853
70 0.19541512429714203
71 0.19541434943675995
72 0.19541355967521667
73 0.1954127848148346
74 0.19541199505329132
75 0.19541120529174805
76 0.19541041553020477
77 0.1954096257686615
78 0.19540883600711823
79 0.19540806114673615
80 0.19540725648403168
81 0.1954064816236496
82 0.19540567696094513
83 0.19540490210056305
84 0.19540411233901978
85 0.1954033225774765
86 0.19540251791477203
87 0.19540172815322876
88 0.19540093839168549
89 0.1954001635313034
90 0.19539937376976013
91 0.19539858400821686
92 0.19539779424667358
93 0.19539698958396912
94 0.19539619982242584
95 0.19539541006088257
96 0.1953946202993393
97 0.19539381563663483
98 0.19539302587509155
99 0.19539223611354828
0.1954497915506363 0.19541155517101289 0.00019563274700913134
运行时间： 3.128347396850586
loss: 0.19539223611354828


In [84]:
netu = FiveNet_sin(n_variables).to(device)
try:
    netu.load_state_dict(torch.load('nets/netu_{}'.format(per)))
except:
    train_fun(netu,cor_train,u_train,N_red_lr=4, epochs=1000,lr=0.01,threshold=0.001)
    torch.save(netu.state_dict(),'net/netu_{}'.format(per))

0 0.7916907072067261
1 0.5617226958274841
2 0.7692429423332214
3 0.5131605863571167
4 0.40592023730278015
5 0.43634575605392456
6 0.39876049757003784
7 0.4189945161342621
8 0.40679800510406494
9 0.3821333646774292
10 0.40306350588798523
11 0.38854727149009705
12 0.37414032220840454
13 0.38149675726890564
14 0.38032594323158264
15 0.37235450744628906
16 0.3707287609577179
17 0.3734738528728485
18 0.3731197714805603
19 0.36944451928138733
20 0.3675444722175598
21 0.3679278492927551
22 0.36883386969566345
23 0.36737093329429626
24 0.3653525412082672
25 0.3641280233860016
26 0.36394643783569336
27 0.3641645610332489
28 0.36354655027389526
29 0.36235693097114563
30 0.36080506443977356
31 0.3593186140060425
32 0.3586260974407196
33 0.35802793502807617
34 0.35646435618400574
35 0.35426855087280273
36 0.35203173756599426
37 0.34964898228645325
38 0.34690961241722107
39 0.3436383605003357
40 0.3394975960254669
41 0.3342214822769165
42 0.3278510272502899
43 0.32055890560150146
44 0.3130899667739

339 0.07546769082546234
340 0.07515724748373032
341 0.07493722438812256
342 0.07483381032943726
343 0.07479844987392426
344 0.07477770000696182
345 0.07475090771913528
346 0.07470405846834183
347 0.07466082274913788
348 0.07465270906686783
349 0.07471325993537903
0.07664546236395836 0.07584119930863381 0.010493289889823215
350 0.07488266378641129
351 0.07517683506011963
352 0.0756552591919899
353 0.07624658197164536
354 0.07697898894548416
355 0.07747037708759308
356 0.07769075036048889
357 0.07731297612190247
358 0.07676772773265839
359 0.07612781971693039
360 0.07575269043445587
361 0.07551675289869308
362 0.07546976208686829
363 0.07543331384658813
364 0.07540934532880783
365 0.07533513009548187
366 0.07527240365743637
367 0.07523094862699509
368 0.07520972192287445
369 0.07518184930086136
370 0.07510031014680862
371 0.07496535032987595
372 0.07481776177883148
373 0.07472658902406693
374 0.07476595789194107
375 0.07497929036617279
376 0.07534752041101456
377 0.07579390704631805
378 

93 0.0727626383304596
94 0.07276137173175812
95 0.07275984436273575
96 0.0727580338716507
97 0.07275652140378952
98 0.07275518029928207
99 0.07275355607271194
0.07444125190377235 0.07279867202043533 0.0220654521697234
100 0.07275185734033585
101 0.07275041192770004
102 0.07274898886680603
103 0.07274734228849411
104 0.07274574041366577
105 0.07274431735277176
106 0.07274279743432999
107 0.07274116575717926
108 0.07273963838815689
109 0.07273818552494049
110 0.07273662090301514
111 0.0727350264787674
112 0.07273352891206741
113 0.07273203134536743
114 0.07273045182228088
115 0.07272889465093613
116 0.07272739708423615
117 0.07272586971521378
118 0.07272429764270782
119 0.07272276282310486
120 0.07272124290466309
121 0.07271969318389893
122 0.07271812856197357
123 0.0727166086435318
124 0.07271507382392883
125 0.07271351665258408
126 0.07271196693181992
127 0.07271043211221695
128 0.07270888984203339
129 0.07270733267068863
130 0.07270579040050507
131 0.0727042406797409
132 0.07270269095

6 0.07233785837888718
7 0.07233767211437225
8 0.0723375603556633
9 0.0723373144865036
10 0.07233705371618271
11 0.07233688235282898
12 0.07233670353889465
13 0.07233646512031555
14 0.07233624160289764
15 0.07233605533838272
16 0.07233583927154541
17 0.0723356157541275
18 0.0723353922367096
19 0.07233519107103348
20 0.07233497500419617
21 0.07233475148677826
22 0.07233452796936035
23 0.07233431190252304
24 0.07233408093452454
25 0.07233385741710663
26 0.07233363389968872
27 0.07233340293169022
28 0.07233317196369171
29 0.0723329409956932
30 0.0723327174782753
31 0.0723324716091156
32 0.07233224809169769
33 0.07233200967311859
34 0.07233177125453949
35 0.07233153283596039
36 0.07233129441738129
37 0.07233105599880219
38 0.07233081012964249
39 0.07233057171106339
40 0.07233033329248428
41 0.07233007997274399
42 0.07232983410358429
43 0.07232958823442459
44 0.07232934236526489
45 0.0723290964961052
46 0.0723288431763649
47 0.0723285973072052
48 0.0723283439874649
49 0.07232809066772461
50 

In [85]:
netv = FiveNet_sin(n_variables).to(device)
try:
    netv.load_state_dict(torch.load('nets/netv_{}'.format(per)))
except:
    train_fun(netv,cor_train,v_train,N_red_lr=4, epochs=1000,lr=0.01,threshold=0.001)
    torch.save(netv.state_dict(),'net/netv_{}'.format(per))

0 1.0883939266204834
1 2.1836607456207275
2 1.3264422416687012
3 1.122421383857727
4 1.0862804651260376
5 1.0247575044631958
6 1.0256319046020508
7 1.0451996326446533
8 1.0291287899017334
9 1.0069435834884644
10 1.006725549697876
11 0.9997871518135071
12 0.9960643649101257
13 0.9961138367652893
14 0.9909200668334961
15 0.9883850812911987
16 0.9861590266227722
17 0.9845784306526184
18 0.9834944605827332
19 0.9777462482452393
20 0.972149133682251
21 0.9689276218414307
22 0.9669878482818604
23 0.9622326493263245
24 0.9533005356788635
25 0.9422192573547363
26 0.9310635328292847
27 0.9145371317863464
28 0.8890498280525208
29 0.8535946011543274
30 0.8060203194618225
31 0.7489563822746277
32 0.7005183100700378
33 0.687569797039032
34 0.7973108887672424
35 1.012893557548523
36 0.640995979309082
37 0.7979336977005005
38 0.6586049795150757
39 0.7075421810150146
40 0.6940465569496155
41 0.6237832307815552
42 0.6495002508163452
43 0.6262044310569763
44 0.5740092992782593
45 0.6090644001960754
46 0

349 0.20489338040351868
0.20450365871191026 0.2031945753097534 0.0064012713043975055
350 0.20377574861049652
351 0.20285852253437042
352 0.2020627111196518
353 0.20146748423576355
354 0.2009170800447464
355 0.20050717890262604
356 0.20026594400405884
357 0.20026551187038422
358 0.20059052109718323
359 0.2011626809835434
360 0.2020740956068039
361 0.2029973268508911
362 0.20417781174182892
363 0.2053254395723343
364 0.2068345844745636
365 0.2091585099697113
366 0.21072907745838165
367 0.2112496793270111
368 0.20840056240558624
369 0.20456883311271667
370 0.20152035355567932
371 0.20011058449745178
372 0.20007802546024323
373 0.20077477395534515
374 0.20161277055740356
375 0.20235873758792877
376 0.20306922495365143
377 0.2036798894405365
378 0.2043464481830597
379 0.2042945921421051
380 0.2036283016204834
381 0.2023700326681137
382 0.20121806859970093
383 0.20054955780506134
384 0.20050452649593353
385 0.20081587135791779
386 0.20123691856861115
387 0.20155484974384308
388 0.20160329341

103 0.19674210250377655
104 0.19673918187618256
105 0.19673635065555573
106 0.1967337727546692
107 0.19673103094100952
108 0.19672808051109314
109 0.19672536849975586
110 0.19672273099422455
111 0.19671988487243652
112 0.1967170387506485
113 0.1967143714427948
114 0.19671161472797394
115 0.19670873880386353
116 0.19670596718788147
117 0.1967032551765442
118 0.19670042395591736
119 0.19669756293296814
120 0.19669482111930847
121 0.19669204950332642
122 0.196689173579216
123 0.19668637216091156
124 0.1966835856437683
125 0.19668075442314148
126 0.19667789340019226
127 0.19667509198188782
128 0.19667229056358337
129 0.19666939973831177
130 0.19666658341884613
131 0.1966637372970581
132 0.1966608762741089
133 0.19665801525115967
134 0.19665516912937164
135 0.19665230810642242
136 0.196649432182312
137 0.1966465711593628
138 0.19664369523525238
139 0.19664080440998077
140 0.19663791358470917
141 0.19663503766059875
142 0.19663214683532715
143 0.19662925601005554
144 0.19662636518478394
145 

In [86]:
cache = diff_NS(netw,cor_train)
w_pred = cache[:,0].reshape(-1,1)
wx = cache[:,1].reshape(-1,1)
wy = cache[:,2].reshape(-1,1)
wxx = cache[:,3].reshape(-1,1)
wxy = cache[:,4].reshape(-1,1)
wyy = cache[:,5].reshape(-1,1)
wt = cache[:,-1].reshape(-1,1)

cache = diff_NS(netu,cor_train)
u_pred = cache[:,0].reshape(-1,1)

cache = diff_NS(netv,cor_train)
v_pred = cache[:,0].reshape(-1,1)

In [87]:
rmse_loss(v_pred,v_clean)

tensor(0.0342, device='cuda:0', dtype=torch.float64)

In [88]:
rmse_loss(-u_pred*wx-v_pred*wy+0.01*(wxx+wyy),wt)

tensor(0.1287, device='cuda:0', dtype=torch.float64)

In [96]:
cache = np.hstack((u_pred,v_pred,w_pred,wx,wy,wxx,wxy,wyy,wt))
cache = random_choice(cache,50000)
LC, exp_list, exp = all0('resultw_{}.txt'.format(per),cache,
                          state=3,tol=0.008,alpha=10**-5,
                          niterations=200, data_points=1000)

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Expressions evaluated per second: 5.390e+05
Head worker occupation: 7.4%
Progress: 1153 / 3000 total iterations (38.433%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           7.337e-01  1.594e+01  y = x₁
3           1.438e-01  8.148e-01  y = (x₃ * -0.65051)
5           1.438e-01  5.347e-05  y = ((x₃ * -0.6506) - 0.0039223)
7           9.345e-02  2.155e-01  y = (((x₄ * 0.0971) - x₀) * x₃)
9           8.974e-02  2.028e-02  y = ((((x₄ * 0.10453) - x₀) * x₃) / 1.0758)
11          3.208e-02  5.144e-01  y = (((x₄ * ((-0.94457 * x₁) / x₃)) - x₀) * x₃)
13          3.159e-02  7.662e-03  y = (((x₄ * (((-0.003196 * x₅) - x₁) / x₃)) - x₀) * x₃)
15          3.026e-02  2.155e-02  y = (((x₄ * ((((x₂ * x₄) * -0.003196) - x₁) / x₃)) - x₀) * x₃)
17          2.148e-02  1.713e-01  y = (((x₄ * (((((-0.03141 * x₇) * -0.31816) / x₄) - x₁) / x₃))...
                                   - 

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.445e+02, tolerance: 5.222e+00
  model = cd_fast.enet_coordinate_descent(


0 tensor(0.1198, device='cuda:0', dtype=torch.float64)


/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.443e+02, tolerance: 5.222e+00
  model = cd_fast.enet_coordinate_descent(


...

Started!

Expressions evaluated per second: 5.390e+05
Head worker occupation: 9.0%
Progress: 1214 / 3000 total iterations (40.467%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.123e+00  1.594e+01  y = -0.049388
3           3.604e-02  1.720e+00  y = (x₁ / -1.0278)
5           2.146e-02  2.592e-01  y = ((0.010921 * x₄) - x₁)
7           1.902e-02  6.036e-02  y = (((x₂ + x₄) * 0.010175) - x₁)
9           1.887e-02  4.051e-03  y = ((0.0094217 * (x₂ + (x₄ - x₀))) - x₁)
11          1.874e-02  3.406e-03  y = ((0.0094217 * ((x₂ + x₁) + (x₄ - x₀))) - x₁)
13          1.869e-02  1.178e-03  y = ((0.0094217 * ((x₂ + (x₄ - x₀)) - (x₀ - x₁))) - x₁)
14          1.868e-02  7.369e-04  y = ((0.0094217 * ((x₂ + x₄) - (x₀ - sin(x₁)))) - x₁)
15          1.867e-02  5.245e-04  y = ((0.0094217 * (((x₂ + x₄) - x₀) - ((x₄ * x₃) * 0.022784)))...
                                   - x₁)
16  

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Expressions evaluated per second: 6.400e+05
Head worker occupation: 7.6%
Progress: 1369 / 3000 total iterations (45.633%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.135e+00  1.594e+01  y = -0.054167
3           2.983e-02  1.819e+00  y = (-0.97379 * x₁)
5           1.504e-02  3.424e-01  y = ((x₃ * 0.0093861) - x₁)
7           1.483e-02  6.993e-03  y = ((0.0092624 * x₃) - (x₁ * 0.98654))
9           1.483e-02  5.991e-05  y = ((0.0092624 * (x₃ + 0.037669)) - (x₁ * 0.98654))
10          1.475e-02  5.595e-03  y = (sin(0.0097698 * (x₁ + x₃)) - x₁)
12          1.474e-02  4.379e-04  y = (((sin(0.0093861 / x₁) * x₃) * x₁) - x₁)
13          1.470e-02  2.201e-03  y = (sin(sin(0.0097698 * (x₁ + x₃))) - x₁)
14          1.455e-02  1.067e-02  y = (((sin(0.0093861 / x₁) * (x₃ + x₁)) * x₁) - x₁)
17          1.447e-02  1.711e-03  y = (sin((sin(0.0093861 / x₁) * (x₃ + x

/home/chr/SI-SR/Symmetry.py:196: RuntimeWarning: invalid value encountered in divide
  print(results)
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Expressions evaluated per second: 5.980e+05
Head worker occupation: 8.6%
Progress: 1280 / 3000 total iterations (42.667%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           9.185e-01  1.594e+01  y = 0.026916
3           9.057e-01  7.024e-03  y = (0.13743 * x₂)
5           8.852e-01  1.143e-02  y = (x₀ * (-0.037337 * x₁))
7           8.769e-01  4.723e-03  y = ((-0.041246 * (x₁ - x₂)) * x₀)
8           7.768e-01  1.213e-01  y = sin((x₂ * x₁) * x₀)
10          7.635e-01  8.636e-03  y = sin(((x₀ * x₁) - 0.13027) * x₂)
12          7.477e-01  1.045e-02  y = (sin(((x₀ * x₁) - 0.13027) * x₂) / 0.67208)
14          6.950e-01  3.652e-02  y = sin((x₂ * (x₀ / 0.28435)) * ((x₀ * x₀) + -0.71612))
16          6.686e-01  1.933e-02  y = (sin((x₂ * (x₀ / 0.28435)) * ((x₀ * x₀) + -0.71612)) * 1.4...
                                  135)
19          6.538e-01  7.493e-03  y = (sin

In [97]:
k = select(LC,0.05)
exp_list[k]

[1 1 1 1 1 1 1 1 1 1 1]
[ 1  1  1  1  1  1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1]


-x1 + 0.009386148*x3